In [3]:
import sys
sys.path.append("src")

from fetchers import fetch_inmet_climate, fetch_ibge_population_density

ModuleNotFoundError: No module named 'fetchers'

In [8]:
#Imports e criação de pasta raw
import os
import pandas as pd

RAW_DIR = "data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

In [9]:
#Download da série de dengue
dengue_url = (
    "https://raw.githubusercontent.com/"
    "valdineyatilio/ProjetoAplicado-IV/"
    "main/A1/DadosDengueCSV.csv"
)
df_dengue = pd.read_csv(dengue_url)
df_dengue.to_csv(f"{RAW_DIR}/dengue.csv", index=False)

In [10]:
#Coleta de clima (INMET)
from fetchers import fetch_inmet_climate

stations = ["A001", "A002"]
df_clima = fetch_inmet_climate(
    station_ids=stations,
    start="2010-01-01",
    end="2025-12-31"
)
df_clima.to_csv(f"{RAW_DIR}/clima_inmet.csv", index=False)

ModuleNotFoundError: No module named 'fetchers'

In [11]:
#Coleta de densidade populacional (IBGE)
from fetchers import fetch_ibge_population_density

municipios = [3106200, 3106309]
df_pop = fetch_ibge_population_density(
    municipio_ids=municipios,
    year=2020
)
df_pop.to_csv(f"{RAW_DIR}/pop_density.csv", index=False)

ModuleNotFoundError: No module named 'fetchers'